In [2]:
# imports
from Testing import test
from Event import *
from llmWrappers.openai_wrapper import askOpenAI, getOpenAIResponse, chat
from llmWrappers.ollama_wrapper import askOllama
from llmWrappers.claude_wrapper import displayClaudeResponse
from llmWrappers.deepseek_wrapper import displayDeepSeekResponse

import json
import pprint
import gradio as gr
import os.path
import datetime
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES=["https://www.googleapis.com/auth/calendar"]

event = Event()
data = event.create_event()
print("prompt in english: ")
print(event.prompt['en'])
print("data in english: ")
print(data['en'])
print("OpenAI: ")
gpt_json = askOpenAI(event.prompt['en'])

print("gpt json: ", gpt_json)

creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "google_desktop_credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
        
try:
    
    service = build("calendar", "v3", credentials=creds)
    # Create event
    event = service.events().insert(calendarId="primary", body=json.loads(gpt_json)).execute()
except HttpError as error:
    print(f"An error occurred: {error}")

start date  2025-06-21
prompt in english: 
meeting with the principal every year at 21 of June
data in english: 
{'summary': 'meeting with the principal', 'start': {'dateTime': '2025-06-21T19:31:00+02:00', 'timeZone': 'UTC'}, 'end': {'dateTime': '2025-06-21T21:31:00+02:00', 'timeZone': 'UTC'}, 'recurrence': ['RRULE:FREQ=YEARLY;BYMONTH=6;BYMONTHDAY=21;UNTIL=20300101T000000Z']}
OpenAI: 
gpt json:  {
  "summary": "meeting with the principal",
  "start": {
    "dateTime": "2025-06-21T09:00:00+02:00",
    "timeZone": "UTC"
  },
  "end": {
    "dateTime": "2025-06-21T10:00:00+02:00",
    "timeZone": "UTC"
  },
  "recurrence": ["RRULE:FREQ=YEARLY;BYMONTH=6;BYMONTHDAY=21"]
}
